# Eksploracja Danych - Projekt
Tomasz Kiljańczyk (136257)

Wojciech Lulek (136280)

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
us_trending_df = pd.read_csv('./data/us_trending_stage_1.csv').head(1000)
us_trending_df

In [ ]:
us_trending_df['thumbnail_link_hires'] = us_trending_df['thumbnail_link'].str.replace('default.jpg', '0.jpg')
us_trending_df['thumbnail_link_hires']

In [ ]:
from notebooks.workers import download_and_save
from multiprocessing import Pool
import os

if not os.path.exists('./images/'):
    os.makedirs('./images/')

data = us_trending_df['thumbnail_link_hires'].unique()
with Pool(processes=os.cpu_count()) as pool:
    for _ in tqdm_notebook(pool.imap(download_and_save, data), total=data.size):
        pass


In [ ]:
from notebooks.workers import extract_color_features
from PIL import Image

image_dataframe = pd.DataFrame(columns=[
    'thumbnail_med_hue',
    'thumbnail_med_saturation',
    'thumbnail_med_value',
    'thumbnail_avg_hue',
    'thumbnail_avg_saturation',
    'thumbnail_avg_value',
    'thumbnail_colorfulness'
])

data = us_trending_df['thumbnail_link_hires'].unique()
results = []

with Pool(processes=os.cpu_count()) as pool:
    for res in tqdm_notebook(pool.imap(extract_color_features, data), total=data.size):
        results.append(res)

color_features_dict = dict(results)

color_features = [color_features_dict[url] for url in us_trending_df['thumbnail_link_hires']]

image_dataframe = image_dataframe.append(color_features, ignore_index=True)
image_dataframe

In [ ]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


for i, url in enumerate(us_trending_df['thumbnail_link_hires'].head(5)):
    file_path = f'./images/{url.split("/")[-2]}.jpg'

    image = Image.open(file_path)

    row = image_dataframe.iloc[i]
    median_hsv_color = (
        round(row['thumbnail_med_hue']),
        round(row['thumbnail_med_saturation']),
        round(row['thumbnail_med_value'])
    )
    avg_hsv_color = (
        round(row['thumbnail_avg_hue']),
        round(row['thumbnail_avg_saturation']),
        round(row['thumbnail_avg_value'])
    )

    dims = (round(image.width * 0.1), image.height)
    median_color_image = Image.new('HSV', dims, color=median_hsv_color).convert('RGB')
    average_color_image = Image.new('HSV', dims, color=avg_hsv_color).convert('RGB')

    image_result_1 = get_concat_h(image, median_color_image)
    image_result_1 = get_concat_h(image_result_1, average_color_image)

    display(image_result_1)

In [ ]:
from notebooks.workers import detect_text

results = []
with Pool(processes=os.cpu_count()) as pool:
    data = us_trending_df['thumbnail_link_hires'].unique()
    for res in tqdm_notebook(pool.imap(detect_text, data), total=data.size):
        results.append(res)

has_text_dict = dict(results)
has_text = [has_text_dict.get(url, False) for url in us_trending_df['thumbnail_link_hires']]

Notatki:
* można spróbować cechy z bounding boxów

In [ ]:
has_text_df = pd.DataFrame(has_text, columns=['has_text'])
has_text_df

In [ ]:
final_df = pd.concat([us_trending_df, image_dataframe], axis=1)
final_df = pd.concat([final_df, has_text_df], axis=1)
final_df

In [ ]:
corrMatrix = final_df.corr(method='spearman')
corrMatrix = corrMatrix.round(4)

mask = np.triu(np.ones_like(corrMatrix, dtype=bool))
np.fill_diagonal(mask, False)

fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corrMatrix, mask=mask, annot=True, fmt='g', ax=ax)
plt.show()

In [ ]:
final_df.to_csv("./us_trending_stage_2_tk.csv", index=False)

# Using ImageAI for object detection

Trained model file is required! Put this into notebooks/models directory. Download link: https://github.com/OlafenwaMoses/ImageAI/releases/download/essentials-v5/resnet50_coco_best_v2.1.0.h5/

Loading pre-trained model

In [ ]:
from imageai.Detection import ObjectDetection
import os

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("models/resnet50_coco_best_v2.1.0.h5")
detector.loadModel()

Detecting objects from thumbnails

In [ ]:
import pickle
import tqdm

dir_name = "images"
dir_filenames = os.listdir(dir_name)
dir_filenames.remove(".gitignore")
images_left = len(dir_filenames)
image_detections = dict()

for filename in tqdm.tqdm(dir_filenames):
    detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename, output_type="array")[1]
    image_detections[filename] = detections

    images_left -= 1
    if images_left % 500 == 0:
        print("Backup")
        with open("data/thumbnails_objects_dict.pkl", "wb") as f:
            pickle.dump(image_detections, f)

Loading detections from pickle file

In [ ]:
import pickle

with open("data/thumbnails_objects_dict.pkl", "rb") as f:
    image_detections = dict(pickle.load(f))

Counting detected objects

In [ ]:
image_objects_detected = dict()
counts = dict()

for key in image_detections.keys():
    image_objects_detected[key] = dict()

    for obj in image_detections[key]:
        name = obj['name']
        image_objects_detected[key][name] = image_objects_detected[key].get(name, 0) + 1
        counts[name] = counts.get(name, 0) + 1

counts = dict(reversed(sorted(counts.items(), key=lambda item: item[1])))
top_popular_objects = list(counts.keys())[:10]

print("counts", counts)
print("10 most common objects:", top_popular_objects)

Creating vector representation and saving it to data/ in CSV format

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(image_objects_detected).transpose()
df = df.fillna(0)
df = df.astype('int32')

df.to_csv("data/thumbnails_objects_vectors.csv")

Notatka: Może wagi wilkością bboxów

może rozpoznawanie emocji na twarzach